In [1]:
import mne

**Import sample**

In [6]:
file = 'models/BIDS/sub-ID000/ses-S005/eeg/sub-ID000_ses-S005_task-Default_run-001_eeg.edf'
raw = mne.io.read_raw_edf(file, preload=True)

# Print basic information about the data
print(raw.info)
print(raw.describe())
print(raw.duration)

Extracting EDF parameters from /Users/andrei/Projects/resonance_installation/notebooks/models/BIDS/sub-ID000/ses-S005/eeg/sub-ID000_ses-S005_task-Default_run-001_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 16127  =      0.000 ...    62.996 secs...
<Info | 8 non-empty values
 bads: []
 ch_names: AF7, TP9, TP10, AF8
 chs: 4 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 128.0 Hz
 meas_date: 2024-03-04 12:19:54 UTC
 nchan: 4
 projs: []
 sfreq: 256.0 Hz
 subject_info: <subject_info | his_id: X, sex: 0, last_name: X>
>
<RawEDF | sub-ID000_ses-S005_task-Default_run-001_eeg.edf, 4 x 16128 (63.0 s), ~513 KiB, data loaded>
ch  name  type  unit        min         Q1     median         Q3        max
 0  AF7   EEG   µV      -539.05     -37.59     -27.82     -17.09     324.70
 1  TP9   EEG   µV      -249.00     -38.55     -31.72     -25.87     192.38
 2  TP10  EEG   µV      -348.14     -53.20     -41.99     -31.23     966.3

In [7]:
channels = raw.info['ch_names']

montage = mne.channels.make_standard_montage('standard_1020')
raw.set_montage(montage)

<RawEDF | sub-ID000_ses-S005_task-Default_run-001_eeg.edf, 4 x 16128 (63.0 s), ~515 KiB, data loaded>

In [ ]:
# ------------------------------
# 2. Preprocessing
# ------------------------------
# Bandpass filter (0.5 - 40 Hz)
raw.filter(0.5, 40., fir_design='firwin')

# Notch filter (remove 50/60 Hz powerline noise)
raw.notch_filter([50, 60])

In [ ]:
# ------------------------------
# 3. Epoching (split into segments)
# ------------------------------
events = mne.make_fixed_length_events(raw, duration=2.0)  # 2-second epochs
epochs = mne.Epochs(raw, events, tmin=0, tmax=2.0, baseline=None, preload=True)

In [ ]:
# ------------------------------
# 4. Feature Extraction
# ------------------------------
def compute_bandpower(data, sf, band):
    psd, freqs = mne.time_frequency.psd_array_welch(data, sf, fmin=band[0], fmax=band[1])
    return np.mean(psd, axis=-1)  # Mean power in the band


def extract_features(epochs):
    features = []
    sfreq = epochs.info['sfreq']  # Sampling frequency
    freqs = {
        'delta': (0.5, 4),
        'theta': (4, 8),
        'alpha': (8, 13),
        'beta': (13, 30),
        'gamma': (30, 40)
    }
    
    for epoch in epochs.get_data():  # Shape: (n_epochs, n_channels, n_times)
        epoch_features = []
        # Bandpower per channel
        for ch_data in epoch:
            for band, (fmin, fmax) in freqs.items():
                power = compute_bandpower(ch_data, sfreq, (fmin, fmax))
                epoch_features.append(power)
        
        # Frontal Alpha Asymmetry (AF8 - AF7)
        alpha_af7 = compute_bandpower(epoch[1], sfreq, (8, 13))  # AF7 = index 1
        alpha_af8 = compute_bandpower(epoch[2], sfreq, (8, 13))  # AF8 = index 2
        faa = np.log(alpha_af8) - np.log(alpha_af7)  # Valence metric
        epoch_features.append(faa)
        
        features.append(epoch_features)
    
    return np.array(features)

from scipy.stats import skew, kurtosis

def extract_time_features(epochs):
    features = []
    for epoch in epochs.get_data():
        mean = np.mean(epoch, axis=1)
        std = np.std(epoch, axis=1)
        features.append(np.concatenate([mean, std]))
    return np.array(features)

X_time = extract_time_features(epochs)

In [ ]:

def extract_labels_from_json(json_file):
    with open(json_file) as f:
        json_data = f.read()
        json_data = json.loads(json_data)
        task_description: str = json_data['TaskDescription']
        task_description = task_description.split(', ')
        task_description = list(map(lambda x: x.split(': '), task_description))
        for x in task_description:
            if x[0] == 'AVG_Valence':
                valence = float(x[1])
            elif x[0] == 'AVG_Arousal':
                arousal = float(x[1])
            elif x[0] == 'AVG_Dominance':
                dominance = float(x[1])

    return valence, arousal, dominance

json_file = "models/BIDS/sub-ID000/ses-S002/eeg/sub-ID000_ses-S002_task-Default_run-001_eeg.json"
valence, arousal, dominance = extract_labels_from_json(json_file)

print(f"Task Description: {valence}, {arousal}, {dominance}")

In [26]:
def process_session(edf_path):
    """Process a single session and return features + labels."""
    # Derive paths
    base_path = edf_path.replace("_eeg.edf", "")
    json_path = f"{base_path}_eeg.json"
    
    # Load data
    raw = mne.io.read_raw_edf(edf_path, preload=True)
    
    # Preprocess
    raw.filter(0.5, 40.)
    raw.notch_filter(50)
    epochs = mne.make_fixed_length_epochs(raw, duration=2.0, preload=True)
    
    # Feature extraction
    X = extract_features(epochs)  # Reuse from Step 4
    valence, arousal, dominance = extract_labels_from_json(json_path)
    return X, valence, arousal, dominance

def batch_process(root_dir="models/BIDS"):
    all_X, all_valence, all_arousal, all_dominance = [], [], [], []
    edf_files = glob(f"{root_dir}/sub-*/ses-*/eeg/*_eeg.edf")
    
    for edf_file in tqdm(edf_files):
        try:
            X, y_valence, y_arousal, y_dominance = process_session(edf_file)
            all_X.append(X)
            all_valence.extend([y_valence] * len(X))
            all_arousal.extend([y_arousal] * len(X))
            all_dominance.extend([y_dominance] * len(X))
        except Exception as e:
            print(f"Failed on {edf_file}: {str(e)}")
    
    # Stack all sessions
    X = np.vstack(all_X)
    print(all_valence)
    y_valence = np.array(all_valence)
    y_arousal = np.array(all_arousal)
    y_dominance = np.array(all_dominance)
    
    return X, y_valence, y_arousal, y_dominance


mne.set_log_level('ERROR')  # Suppress MNE warnings
X, y_valence, y_arousal, y_dominance = batch_process()
np.savez("NeuroSense_all_sessions.npz", X=X, y_valence=y_valence, y_arousal=y_arousal, y_dominance=y_dominance)

100%|██████████| 1198/1198 [02:35<00:00,  7.69it/s]

[3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 3.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 4.3333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 3.9333, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667, 6.6667,

In [19]:
mne.set_log_level('ERROR')  # Suppress MNE warnings
X, y_valence, y_arousal, y_dominance = batch_process()
np.savez("NeuroSense_all_sessions.npz", X=X, y_valence=y_valence, y_arousal=y_arousal, y_dominance=y_dominance)

100%|██████████| 1198/1198 [02:31<00:00,  7.90it/s]


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten
data = np.load("NeuroSense_all_sessions.npz")
X = data["X"]  # Features: [n_epochs, n_features]
y_valence = data["y_valence"]  # Continuous scores (1-9)
y_arousal = data["y_arousal"]

# Binarize valence/arousal (threshold=5)
y_valence_bin = (y_valence >= 5).astype(int)  # 1=High, 0=Low
y_arousal_bin = (y_arousal >= 5).astype(int)

# Russell's 4-quadrant labels (0-3)
y_quadrant = 2 * y_arousal_bin + y_valence_bin  # 0=LAHV, 1=HALV, 2=LAHV, 3=HAHV
X_train, X_test, y_train, y_test = train_test_split(X, y_valence_bin, train_size=0.8, random_state=48)

inputs = tf.keras.Input(shape=(X_train.shape[1],1))

gru = tf.keras.layers.GRU(256, return_sequences=True)(inputs)
flat = Flatten()(gru)
outputs = Dense(3, activation='softmax')(flat)

model = tf.keras.Model(inputs, outputs)

model.summary()


AttributeError: `np.float_` was removed in the NumPy 2.0 release. Use `np.float64` instead.